In [1]:
import random
import numpy as np
from numpy.random import choice
import pandas as pd
from random import randint as rand
import copy

In [15]:
mutationRate = 0.01
totalPopulation = 10000
crossOver = 0.5

In [16]:
alpha_list = [x for x in range(0,8)]

In [18]:
def checkInc_x_Inc_y(x, y, randomData):
    isAttack = 0
    while x <=7 and y <= 7:
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            break
        else:
            x += 1
            y += 1
    return isAttack

In [19]:
def checkInc_x_Dec_y(x, y, randomData):
    isAttack = 0
    while x <=7 and y >= 0:
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            break
        else:
            x += 1
            y -= 1
    return isAttack

In [20]:
def checkDec_x_Inc_y(x, y, randomData):
    isAttack = 0
    while x >= 0 and y <= 7:
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            break
        else:
            x -= 1
            y += 1
    return isAttack

In [21]:
def checkDec_x_Dec_y(x, y, randomData):
    isAttack = 0
    while x >= 0 and y >= 0:
        #print('Checking if any queen exists in x : '+str(x) + 'and y : '+str(y))
        isAttack = y == randomData[x]
        #print(isAttack)
        if isAttack == 1:
            break
        else:
            x -= 1
            y -= 1
    return isAttack

In [22]:
def getFitnessScore(randomData):
    fitnessScore = 0
    #Iterate through each of the queens position and check if it attacks any other queen
    for idx, queen in enumerate(randomData):
        # Check if there is any other queen within the same Row 
        # print('Checking Queen Position x : '+str(idx) +'and y : ' +str(queen))
        if randomData.count(queen) == 1:
            # Check Diagonality constraint of this queen with others
            isAttack_Inc_x_Inc_y = checkInc_x_Inc_y(idx+1,queen+1,randomData)
            isAttack_Inc_x_Dec_y = checkInc_x_Dec_y(idx+1,queen-1,randomData)
            isAttack_Dec_x_Inc_y = checkDec_x_Inc_y(idx-1,queen+1,randomData)
            isAttack_Dec_x_Dec_y = checkDec_x_Dec_y(idx-1,queen-1,randomData)
            if (isAttack_Inc_x_Inc_y == 0 and isAttack_Inc_x_Dec_y == 0 and isAttack_Dec_x_Inc_y == 0 and isAttack_Dec_x_Dec_y == 0):
                fitnessScore += 1
        # print('Queen Position for Column '+str(idx) +'---' +str(queen) +' and its fitnessScore is '+str(fitnessScore))
    return fitnessScore

In [23]:
populationData = []
fitnessData = []
secure_random = random.SystemRandom()
for outloop in range(totalPopulation):
  randomData = []
  fitnessScore = 0
  for inloop in range(8):
    selectedData = secure_random.choice(alpha_list)
    randomData.append(selectedData)
  #randomData = [1,4,3,0,2,5,6,0]
  populationData.append(randomData)
  fitnessScore = getFitnessScore(randomData)
  fitnessData.append(fitnessScore)
probabilityDist = []
for outloop in range(totalPopulation):
  probabilityDist.append(fitnessData[outloop]/8)
probDataFrame = pd.DataFrame({'String':populationData,'FitnessScore':fitnessData,'Probability':probabilityDist})
probDataFrame = probDataFrame.sort_values(['Probability'],ascending=False)
probDataFrame = probDataFrame.reset_index(drop=True)

In [25]:
 loop = 1
while True:
    #for idx in range(len(populationData)):
    #    print('Element ----', idx, probDataFrame[idx:idx+1]["String"].values[0], 'and fitness Score is ',
    #          getFitnessScore(probDataFrame[idx:idx+1]["String"].values[0]))
    for idx in range(len(populationData)):
        draw = []
        draw.append(copy.deepcopy(probDataFrame.iloc[idx]["String"]))
        draw.append(copy.deepcopy(probDataFrame.iloc[idx+1]["String"]))
        if draw[0] == draw[1]:
            continue
        else:
            break
    if getFitnessScore(draw[0]) == 8 or getFitnessScore(draw[1]) == 8:
        print(draw[0], ' ', draw[1])
        break
    for cross in range(6):
        if abs(draw[0][cross] - draw[1][cross + 1]) < 2:
            draw[0][cross], draw[1][cross] = draw[1][cross], draw[0][cross]
        if abs(draw[1][cross] - draw[0][cross + 1]) < 2:
            draw[1][cross], draw[0][cross] = draw[0][cross], draw[1][cross]
    child1 = draw[0]
    child2 = draw[1]
    bound = 3
    leftsizeIndex = rand(0, bound)
    rightsizeIndex = rand(bound + 1, 7)
    newchild1 = []
    newchild2 = []
    for dna in child2:
        if dna not in child2:
            newchild2.append(dna)
    for i in range(8):
        if i not in newchild2:
            newchild2.append(i)
    for dna in child1:
        if dna not in newchild1:
            newchild1.append(dna)
    for i in range(8):
        if i not in newchild1:
            newchild1.append(i)
    newchild1[leftsizeIndex], newchild1[rightsizeIndex] = newchild1[rightsizeIndex], newchild1[leftsizeIndex]
    newchild2[leftsizeIndex], newchild2[rightsizeIndex] = newchild2[rightsizeIndex], newchild2[leftsizeIndex]
    populationData.append(newchild1)
    populationData.append(newchild2)
    fitnessData = []
    totalPopulation = len(populationData)
    for outloop in range(totalPopulation):
        fitnessScore = getFitnessScore(populationData[outloop])
        fitnessData.append(fitnessScore)
    probabilityDist = []
    for outloop in range(totalPopulation):
        probabilityDist.append(fitnessData[outloop] / 8)
    probDataFrame = pd.DataFrame(
        {'String': populationData, 'FitnessScore': fitnessData, 'Probability': probabilityDist})
    probDataFrame = probDataFrame.sort_values(['Probability'], ascending=False, ignore_index=True)
    print('Generation ', loop, ' ', ' Average Fitness Score ', probDataFrame["FitnessScore"].mean(), ' ', newchild1,
              ' ',
              getFitnessScore(newchild1))
    loop += 1

Generation  1    Average Fitness Score  0.7966406718656269   [4, 1, 5, 3, 2, 0, 6, 7]   3
Generation  2    Average Fitness Score  0.7969812075169932   [1, 7, 2, 3, 6, 4, 0, 5]   5
Generation  3    Average Fitness Score  0.7970217869278433   [3, 7, 6, 0, 5, 2, 4, 1]   2
Generation  4    Average Fitness Score  0.7968625099920064   [0, 1, 7, 4, 3, 6, 5, 2]   0
Generation  5    Average Fitness Score  0.7969030969030969   [2, 7, 3, 4, 0, 1, 6, 5]   2
Generation  6    Average Fitness Score  0.7970435477427088   [4, 6, 2, 3, 7, 1, 0, 5]   3
Generation  7    Average Fitness Score  0.7969842220890753   [4, 6, 5, 1, 7, 0, 2, 3]   1
Generation  8    Average Fitness Score  0.7974241214057508   [2, 5, 1, 4, 0, 3, 6, 7]   6
Generation  9    Average Fitness Score  0.7973647434617688   [4, 7, 3, 1, 6, 0, 2, 5]   1
Generation  10    Average Fitness Score  0.7974051896207585   [2, 7, 6, 3, 4, 1, 0, 5]   2
Generation  11    Average Fitness Score  0.7975454001197366   [2, 6, 1, 4, 0, 3, 7, 5]   3
Generati